Importando as bibliotecas e o vídeo

In [1]:
import moviepy.editor as mpe
import numpy as np

video = mpe.VideoFileClip("Original.mp4")

Invertendo a imagem a cada 20s

In [ ]:
# Função para inverter a imagem horizontalmente
def inverter_frame(frame):
    return np.flip(frame, axis=1)  # Inverter horizontalmente

# Função para processar o vídeo a cada 20 segundos
def processar_video(clip):
    def inverter_a_cada_20s(get_frame, t):
        # Verificar se está em um intervalo de 20 segundos
        if int(t) // 20 % 2 == 1:
            frame = get_frame(t)
            return inverter_frame(frame)
        else:
            return get_frame(t)
    
    return clip.fl(inverter_a_cada_20s)

# Processar o vídeo
video_invertido = processar_video(video)

# Salvar o vídeo processado
video_invertido.write_videofile("video_invertido.mp4", codec="libx264")

Diminuindo o volume a cada 30s

In [ ]:
def volume_function(t, audio_shape):

    mask = t >= duracao_total - 10
    intervals = t // 30
    factors = 1 - 0.1 * intervals
    factors[mask] = 0.0
    return np.tile(np.maximum(factors, 0.0), (audio_shape[1], 1)).T

def aplicar_volume(get_audio_frame, t):
    
    audio_frame = get_audio_frame(t)
    fator = volume_function(t, audio_frame.shape)
    # Verificando se os tipos de dados são compatíveis
    if audio_frame.dtype != fator.dtype:
        fator = fator.astype(audio_frame.dtype)
    return audio_frame * fator

# Carregar o vídeo
video_invertido_com_audio = mpe.VideoFileClip("video_invertido.mp4")
duracao_total = video_invertido_com_audio.duration

# Aplicar a redução de volume
audio_reduzido = video_invertido_com_audio.audio.fl(aplicar_volume)

# Combinar o vídeo e o áudio
video_final = video_invertido_com_audio.set_audio(audio_reduzido)

# Salvar o vídeo final
video_final.write_videofile("video_final.mp4", codec="libx264")

In [ ]:
parte1 = video_final.subclip(0,60)
parte2 = video_final.subclip(60,80)
parte3 = video_final.subclip(80)
partes = mpe.concatenate_videoclips([parte1,parte3,parte2])

partes.write_videofile('video_cortado.mp4')